In [109]:
import pandas as pd

train = pd.read_csv(r"F:\Machine_learning\HackerEarth\train.csv")
test = pd.read_csv(r"F:\Machine_learning\HackerEarth\test.csv")

train.columns
train.dtypes
# ALL features are either float or integers. and the output is categorical into 4 values. which will be mapped to 1,2,3,4
train.Violations.value_counts() # this is categorical Variable
train.Accident_Type_Code.value_counts() # This is a Categorical variable


train = pd.get_dummies(train, columns =['Accident_Type_Code','Violations'])
test = pd.get_dummies(test, columns =['Accident_Type_Code','Violations'])

train.columns
train = train[['Accident_ID', 'Safety_Score', 'Days_Since_Inspection',
       'Total_Safety_Complaints', 'Control_Metric', 'Turbulence_In_gforces',
       'Cabin_Temperature', 'Max_Elevation', 'Adverse_Weather_Metric',
        'Accident_Type_Code_2',
       'Accident_Type_Code_3', 'Accident_Type_Code_4', 'Accident_Type_Code_5',
       'Accident_Type_Code_6', 'Accident_Type_Code_7', 
       'Violations_1', 'Violations_2', 'Violations_3', 'Violations_4',
       'Violations_5','Severity']] # 'Accident_Type_Code_1','Violations_0',
# 'Severity' is sent to last columns as that is the output. Accident_ID is made first columns since thats the output format.

train.Severity.value_counts().sum() # 10000 values
train.shape # 10000 x 23
# to confirm the values in the output and shape of the train data.
train.Severity.value_counts() # Distribution of the data along output types.

# Mapping all 4 types to numeric values. 
train.Severity = train.Severity.map({'Highly_Fatal_And_Damaging':0,
                                     'Significant_Damage_And_Serious_Injuries':1,
                                     'Minor_Damage_And_Injuries':2,
                                     'Significant_Damage_And_Fatalities':3},)
test.columns
# Adding Accident_ID as the first column to the test data.
test = test[['Accident_ID','Safety_Score', 'Days_Since_Inspection', 'Total_Safety_Complaints',
       'Control_Metric', 'Turbulence_In_gforces', 'Cabin_Temperature',
       'Max_Elevation', 'Adverse_Weather_Metric',
        'Accident_Type_Code_2', 'Accident_Type_Code_3',
       'Accident_Type_Code_4', 'Accident_Type_Code_5', 'Accident_Type_Code_6',
       'Accident_Type_Code_7',  'Violations_1', 'Violations_2',
       'Violations_3', 'Violations_4', 'Violations_5']] # 'Accident_Type_Code_1','Violations_0',

train.shape, test.shape # one extra column in train data which is the output.
# since ID's are not the part of the data input, they are saved into different columns and later will mapped onto
# the output data for submission.
Acc_ID_Train = train.Accident_ID
Acc_ID_Test = test.Accident_ID
# Dropping the Accident ID
train = train.iloc[:,1:]
test = test.iloc[:,1:]


# Create X_train, y_train and we have test data withour output.
X = train.iloc[:,0:-1]
y = train.iloc[:,[-1]] # severityb

# X.shape, test.shape


# X, y &#10132; X_train, y_train, X_test, y_test

In [110]:
from sklearn import datasets, linear_model, metrics 
# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0,max_depth=85, max_leaf_nodes = 300)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)


# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)


In [111]:
cm

array([[843,  26,  18,  18],
       [ 27, 791,  20,   5],
       [ 30,  21, 699,   8],
       [ 13,   6,  14, 461]], dtype=int64)

In [112]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 100)
accuracies.mean()
# accuracies.std()


0.9313092928128589

In [113]:
cm
metrics.accuracy_score(y_test, y_pred)*100

93.13333333333334

In [79]:
y_pred_sub = classifier.predict(test)

y_pred_sub

array([0, 3, 1, ..., 1, 1, 0], dtype=int64)

In [70]:

sub = pd.DataFrame(Acc_ID_Test)
sub['Severity'] = y_pred_sub.tolist()
sub.Severity = sub.Severity.map({0:'Highly_Fatal_And_Damaging',
                   1:'Significant_Damage_And_Serious_Injuries',
                   2:'Minor_Damage_And_Injuries',
                   3:'Significant_Damage_And_Fatalities'})
sub.to_csv(r"F:\Machine_learning\HackerEarth\sample_submission_DTR.csv",index = None, header=True)

# X, y as Training Data (No Spliting)

In [80]:
from sklearn import datasets, linear_model, metrics 
# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0, max_leaf_nodes = 300) #max_depth=85,
classifier.fit(X, y)

# Predicting the Test set results
y_pred = classifier.predict(X)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_pred)

In [81]:
cm

array([[2976,   18,   14,   41],
       [  18, 2682,   24,    5],
       [  32,   10, 2478,    7],
       [   7,   11,    1, 1676]], dtype=int64)

In [82]:
cm
metrics.accuracy_score(y, y_pred)*100

98.11999999999999

In [86]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X, y = y, cv = 100)
print("Accuracy: %0.2f (+/- %0.3f)" % (accuracies.mean(), accuracies.std() * 2))


Accuracy: 0.94 (+/- 0.043)


In [84]:
accuracies.std()

0.021513672803785824

In [ ]:
y_pred_sub = classifier.predict(test)


sub = pd.DataFrame(Acc_ID_Test)
sub['Severity'] = y_pred_sub.tolist()
sub.Severity = sub.Severity.map({0:'Highly_Fatal_And_Damaging',
                   1:'Significant_Damage_And_Serious_Injuries',
                   2:'Minor_Damage_And_Injuries',
                   3:'Significant_Damage_And_Fatalities'})
sub.to_csv(r"F:\Machine_learning\HackerEarth\sample_submission_DTR.csv",index = None, header=True)

# Outliers Removed

In [100]:
# df = pd.DataFrame(np.random.randn(100, 3))
import numpy as np
from scipy import stats
train_OL = train[(np.abs(stats.zscore(train)) < 3).all(axis=1)]

In [101]:
train_OL.shape

(7384, 22)

In [102]:
# Create X_train, y_train and we have test data withour output.
X_OL = train_OL.iloc[:,0:-1]
y_OL = train_OL.iloc[:,[-1]] # severityb


In [103]:
from sklearn import datasets, linear_model, metrics 
# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_OL, y_OL, test_size = 0.3, random_state = 0)


# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0,max_depth=85, max_leaf_nodes = 300)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)


# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[633,  24,  20,  15],
       [ 21, 595,  22,  13],
       [ 20,  21, 451,  11],
       [ 17,   6,  19, 328]], dtype=int64)

In [104]:
metrics.accuracy_score(y_test, y_pred)*100

90.568592057761734

In [108]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: %0.2f (+/- %0.3f)" % (accuracies.mean(), accuracies.std() * 2))


Accuracy: 0.91 (+/- 0.025)
